In [1]:
from pyscf import gto, scf, mp, cc, dft
import numpy as np
import ase
import ase.visualize

from matplotlib import pyplot as plt

import pandas as pd
import os
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

import explore_qml_data as eqd
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp')
import prepare_calculations
import glob

In [2]:
def atoms_wrapper(amon):
    """
    item of amon dict
    """
    coords = []
    elements = ''
    for line in amon[2:]:
        el, x, y, z = line.split()
        coords.append([float(x),float(y),float(z)])
        elements += el
    
    atoms_object = ase.Atoms(elements, coords)
    return(atoms_object)

def get_e_npbc(nuc_charges, positions):
    atom = []
    for a, c in zip(nuc_charges, positions):
        atom.append([int(a), tuple(c)])

    mol = gto.Mole()
    mol.verbose = 0
    #mol.output = 'H2_{}.log'.format(d)
    mol.atom = atom
    mol.basis = 'def2tzvp'
    mol.build()

    # PBE
    mdft = dft.RKS(mol)
    mdft.xc = 'pbe'
    e_pbe = mdft.kernel()
    if mdft.converged:
        print('converged')
    else:
        print('not converged')
    return(e_pbe)

In [3]:
def align_molecule(molecule, n_plane, p1, p2, s):
    """
    rotates molecule such that the plane formed by the atoms at positions p1, p2, s is orthogonal to n_plane
    """
    v1 = molecule.get_positions()[s] - molecule.get_positions()[p1]
    v2 = molecule.get_positions()[s] - molecule.get_positions()[p2]
    n = np.cross(v1, v2)/np.linalg.norm(np.cross(v1, v2))
    molecule.rotate(n, n_plane)

def wrapper_geometry(atoms, atoms_ref, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = prepare_calculations.get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = prepare_calculations.get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher

    # shift/rotate molecule to be in plane of grid points
    centroid_initial = np.mean(atoms_ref.get_positions(), axis=0)
    shift = np.array([boxsize,boxsize,boxsize])/2 - centroid_initial
    atoms.set_positions(atoms.get_positions() + shift)
    
    align_molecule(atoms, np.array([0,0,1]), 1, 4, 2)
    pos_z = atoms.get_positions()[2,2]
    lv = boxsize/num_gpts
    final_shift = np.array([0,0,int(num_gpts/2)*lv - pos_z])
    atoms.set_positions(atoms.get_positions() + final_shift)
    
    coords_final = atoms.get_positions()

    # get correct lambda value
    lambda_values = np.array([0.4, 0.6, 0.8, 1.0])
    for lam_val in lambda_values:
        new_lambda, scaled_ve = prepare_calculations.get_lambda(lam_val, num_ve)
        # scaled_ve is number of electrons added from pseudopotential file, the remaining electrons must be added in form of a negative charge
        charge = scaled_ve - num_ve # write input

        # create directory if necessary
        if scaled_ve < 10:
            scaled_ve_str = '0'+str(scaled_ve)
        else:
            scaled_ve_str = str(scaled_ve)
        lambda_path = os.path.join(compound_path, f've_{scaled_ve_str}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if new_lambda > 0.5:
            prepare_calculations.write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, template_inp, debug = False)
        else:
            prepare_calculations.write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, template_inp_small_lambda, debug = False)

        # generate pp-files
        prepare_calculations.write_pp_files_compound(atom_symbols, new_lambda, lambda_path, pp_dir, pp_type)

In [4]:
amons_dict = uqm.load_obj('/home/misa/datasets/amons_qm9_11k/unique_amons_dict')
butanol_text = amons_dict['CCCCO']
butanol = atoms_wrapper(butanol_text)

In [ ]:
#ase.visualize.view(butanol, viewer='x3d')

# Bonds

In [ ]:
def distort_bond(basepath, comp_name, distant_indices, distortions, group_ind, molecule):
    mols = []
    for i, d in enumerate(distortions):
        mol_distorted = molecule.copy()
        mol_distorted.set_distance(distant_indices[0], distant_indices[1], d, fix=0, add = True, indices = group_ind)
        path = os.path.join(basepath, f'dist_{d}')
        filename = os.path.join(path, f'{comp_name}_dist_{d}.xyz')
        os.makedirs(path, exist_ok=True)
        print(f'stretching = {mol_distorted.get_distance(distant_indices[0], distant_indices[1])}')
        ase.io.write(filename, mol_distorted, format='xyz')

        mols.append(mol_distorted)
#         if d < 0 and distortions[i+1]>0:
#             mols.append(molecule)
    return(mols)

In [ ]:
basepath = '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch/butanol'
comp_name = 'butanol'
distant_indices = [2,4]
distortions = np.array([-0.5  , -0.375, -0.25 ,  0.   ,  0.25 ,  0.5  ,  0.75 ,  1.   ])
group_ind = [4,14]
molecules = distort_bond(basepath, comp_name, distant_indices, distortions, group_ind, butanol)

In [ ]:
results = []
i = 0
for d, e in zip([-0.5,0,0.5]+[-0.25, 0.0,0.25, 0.75, 1.0]+[-0.375], e_pbe+e_pbe2+e_pbe3):
    if i != 1:
        results.append([d,e])
    i += 1
results.sort()
results = np.array(results)

In [ ]:
plt.plot(results[:,0], results[:,1], '-o')


plt.xlabel(r'$\Delta d_{\rm{eq}}$')
plt.ylabel('energy (Ha)')

In [ ]:
with open('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch//butanol/butanol_npbc_energies.txt', 'w') as f:
    for d, e in zip(results[:,0], results[:,1]):
        f.write(f'{d} {e}\n')

## Prepare input files - Bonds

In [ ]:
# paths to atoms objects
paths = glob.glob('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/bond_stretch//butanol/dist_*/*.xyz')
paths.sort()

# define parameters
pp_dir = '/home/misa/PP_LIBRARY'
pp_type = '_GH_PBE'
template_inp = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe.inp'
template_inp_small_lambda = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe_small_lambda.inp'

# generate input files for all atoms objects
for p in paths:
    atoms = ase.io.read(p)
    print(atoms.get_distance(2,4))
    compound_path = os.path.dirname(p)
    wrapper_geometry(atoms, butanol, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda)

# Angles

In [ ]:
def distort_angles(angle_indices, basepath, comp_name, distortions, group_ind, molecule):
    mols = []
    for i, d in enumerate(distortions):
        mol_distorted = molecule.copy()
        mol_distorted.set_angle(angle_indices[0], angle_indices[1], angle_indices[2], angle=d, add = True, indices = group_ind)
        #basepath = '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/water'
        path = os.path.join(basepath, f'ang_{d}')
        filename = os.path.join(path, f'{comp_name}_ang_{d}.xyz')
        os.makedirs(path, exist_ok=True)
        print(f'angle = {mol_distorted.get_angle(angle_indices[0], angle_indices[1],angle_indices[2])}')
        ase.io.write(filename, mol_distorted, format='xyz')

        mols.append(mol_distorted)
        if d < 0 and distortions[i+1]>0:
            mols.append(molecule)
    return(mols)

In [ ]:
# CCO angle is described by indices 124, H of Oh group is index 14

In [ ]:
basepath = '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/butanol'
comp_name = 'butanol'
distortions = np.linspace(-50, 50, 9)
group_ind = [4, 14]
molecules = distort_angles([1,2,4], basepath, comp_name,distortions, group_ind, butanol)

In [ ]:
e_pbe = []
for m in molecules:
    e_pbe.append(get_e_npbc(m.get_atomic_numbers(), m.get_positions()))

In [ ]:
plt.plot(distortions, e_pbe, '-o')
plt.xlabel('angle')
plt.ylabel('energy (Ha)')

In [ ]:
with open('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/butanol/butanol_npbc_energies.txt', 'w') as f:
    for d, e in zip(distortions, e_pbe):
        f.write(f'{d} {e}\n')

## Prepare input files - Angles

In [ ]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp')
import prepare_calculations

In [ ]:
# paths to atoms objects
paths = glob.glob('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/butanol_plane/ang_*/*.xyz')
paths.sort()

# define parameters
pp_dir = '/home/misa/PP_LIBRARY'
pp_type = '_GH_PBE'
template_inp = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe.inp'
template_inp_small_lambda = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe_small_lambda.inp'
butanol_ref = ase.io.read(paths[4])
# generate input files for all atoms objects
for p in paths:
    atoms = ase.io.read(p)
    print(atoms.get_angle(1,2,4))
    compound_path = os.path.dirname(p)
    wrapper_geometry(atoms, butanol_ref, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda)

In [ ]:
# # paths to atoms objects
# paths = glob.glob('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/butanol_plane/ang_*/*.xyz')
# paths.sort()

# # define parameters
# pp_dir = '/home/misa/PP_LIBRARY'
# pp_type = '_GH_PBE'
# template_inp = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe.inp'
# template_inp_small_lambda = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe_small_lambda.inp'

# # generate input files for all atoms objects
# for p in paths:
#     atoms = ase.io.read(p)
#     print(atoms.get_angle(1,2,4))
#     compound_path = os.path.dirname(p)
#     prepare_calculations.wrapper_ase(atoms, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda)

# Dihedrals

In [1]:
def distort_dihedrals(dihedral_indices, basepath, comp_name, dihedral_angle, group_ind, molecule):
    mols = []
    for i, d in enumerate(dihedral_angle):
        mol_distorted = molecule.copy()
        mol_distorted.set_dihedral(dihedral_indices[0], dihedral_indices[1], dihedral_indices[2], dihedral_indices[3], angle=d, indices = group_ind)
        #basepath = '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/angles/water'
        path = os.path.join(basepath, f'dihe_{d}')
        filename = os.path.join(path, f'{comp_name}_dihe_{d}.xyz')
        os.makedirs(path, exist_ok=True)
        print(f'dihedral angle = {mol_distorted.get_dihedral(dihedral_indices[0], dihedral_indices[1], dihedral_indices[2], dihedral_indices[3])}')
        ase.io.write(filename, mol_distorted, format='xyz')
        mols.append(mol_distorted)
    return(mols)

In [8]:
amons_dict = uqm.load_obj('/home/misa/datasets/amons_qm9_11k/unique_amons_dict')
butanol_text = amons_dict['CCCCO']
butanol = atoms_wrapper(butanol_text)

In [9]:
dihedral_ind = [4,2,1,0]
dihedral_angles = np.arange(0, 210, 30)
group_indices = [0,3,5,6,7,8,11,12,13]
basepath = '/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol'
comp_name = 'butanol'
geometries = distort_dihedrals(dihedral_ind, basepath, comp_name, dihedral_angles, group_indices, butanol)

dihedral angle = 0.0
dihedral angle = 30.000000000000043
dihedral angle = 60.00000000000006
dihedral angle = 90.00000000000007
dihedral angle = 120.00000000000006
dihedral angle = 150.00000000000009
dihedral angle = 180.0


In [10]:
# paths to atoms objects
paths = glob.glob('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_*/*.xyz')
paths.sort()

# define parameters
pp_dir = '/home/misa/PP_LIBRARY'
pp_type = '_GH_PBE'
template_inp = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe.inp'
template_inp_small_lambda = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe_small_lambda.inp'
butanol_ref = ase.io.read(paths[-1])
# generate input files for all atoms objects
test = []
for p in paths:
    atoms = ase.io.read(p)
    print(atoms.get_dihedral(4,2,1,0))
    compound_path = os.path.dirname(p)
    wrapper_geometry(atoms, butanol_ref, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda)

0.0
119.99999999570622
149.99999987068503
180.0
29.999999959312056
60.00000004566035
90.00000016634651


In [ ]:
energies = []
for g in geometries:
    energies.append(get_e_npbc(g.get_atomic_numbers(), g.get_positions()))

In [ ]:
plt.plot(dihedral_angles, energies)

In [ ]:
dihedral_energies = np.array([dihedral_angles, energies])
np.savetxt('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihedral_energies.txt', dihedral_energies.T, delimiter='\t', header='angle energy')

In [ ]:
#ase.visualize.view(geometries[6], viewer='x3d')

## Prepare input files - Dihedrals

In [7]:
import glob
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp')
import prepare_calculations

In [8]:
# paths to atoms objects
paths = glob.glob('/home/misa/projects/atomic-energies/data/ueg_reference/amons/geometry/dihedrals/butanol/dihe_*/*.xyz')
paths.sort()

# define parameters
pp_dir = '/home/misa/PP_LIBRARY'
pp_type = '_GH_PBE'
template_inp = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe.inp'
template_inp_small_lambda = '/home/misa/projects/atomic-energies/data/cpmd_params_template_pbe_small_lambda.inp'
butanol_ref = ase.io.read(paths[-1])
# generate input files for all atoms objects
for p in paths:
    atoms = ase.io.read(p)
    print(atoms.get_dihedral(4,2,1,0))
    compound_path = os.path.dirname(p)
    wrapper_geometry(atoms, butanol_ref, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda)

0.0
119.99999999570622
149.99999987068503
180.0
29.999999959312056
60.00000004566035
90.00000016634651
